In [51]:
import numpy as np
import pandas as pd
import pickle
import sys
import json
import gzip

from torch.optim import Adam
from skorch import NeuralNetRegressor

sys.path.insert(0, '..')
from src import model
from src.model.utility import RMSELoss

sys.modules['model'] = model

In [114]:
data = np.load('../datasets/processed/UIR_5Core.npz')
x = data['x']
y = data['y']
uir = np.concatenate((x, np.expand_dims(y, 1).astype(int)), axis = 1)
U_size = data['U_size']
I_size = data['I_size']
G_b = data['G_b']

data_2 = np.load('../datasets/processed/UI_lists.npz', allow_pickle = True)
U_list = data_2['u']
I_list = data_2['i']

item_desc_path = '../datasets/raw/meta_Toys_and_Games.json.gz'
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

item_desc_df = getDF(item_desc_path)

In [134]:
BATCH_SIZE = 64
NUM_FEATURES = 10
LEARNING_RATE = 0.0005
EPOCHS = 15
DECAY = 1e-3
DROPOUT = 0.4

mf_model = MF_Bias(U_size, I_size, NUM_FEATURES, G_b, DROPOUT)
loss_fn = RMSELoss()
optimizer = Adam

regressor = NeuralNetRegressor(
    mf_model,
    criterion = loss_fn,
    optimizer = optimizer,
    optimizer__param_groups = [
        ('user_m.weight', {'weight_decay': DECAY}),
        ('item_m.weight', {'weight_decay': DECAY})
    ],
    optimizer__lr = LEARNING_RATE,
    batch_size = BATCH_SIZE,
    max_epochs = EPOCHS
)

with open('../results/model_bs64K10LR0.0005E15D0.001DR0.4.pkl', 'rb') as f:
    mf_model = pickle.load(f)

In [62]:
item_desc_df

,asin,description,title,price,salesRank,imUrl,brand,categories,related
0,0000191639,"Three Dr. Suess' Puzzles: Green Eggs and Ham, ...",Dr. Suess 19163 Dr. Seuss Puzzle 3 Pack Bundle,37.12,{'Toys & Games': 612379},http://ecx.images-amazon.com/images/I/414PLROX...,Dr. Seuss,"[[Toys & Games, Puzzles, Jigsaw Puzzles]]",NaN
1,0005069491,NaN,Nursery Rhymes Felt Book,NaN,{'Toys & Games': 576683},http://ecx.images-amazon.com/images/I/51z4JDBC...,NaN,[[Toys & Games]],NaN
2,0076561046,Learn Fractions Decimals Percents using flash ...,Fraction Decimal Percent Card Deck,NaN,{'Toys & Games': 564211},http://ecx.images-amazon.com/images/I/51ObabPu...,NaN,"[[Toys & Games, Learning & Education, Flash Ca...",{'also_viewed': ['0075728680']}
3,0131358936,"New, Sealed. Fast Shipping with tracking, buy ...",NaN,36.22,{'Software': 8080},http://ecx.images-amazon.com/images/I/51%2B7Ej...,NaN,"[[Toys & Games, Learning & Education, Mathemat...","{'also_bought': ['0321845536', '0078787572'], ..."
4,0133642984,NaN,Algebra 2 California Teacher Center,731.93,{'Toys & Games': 1150291},http://ecx.images-amazon.com/images/I/51VK%2BL...,Prentice Hall,"[[Toys & Games, Learning & Education, Mathemat...",NaN
...,...,...,...,...,...,...,...,...,...
336067,B00LOYLHOE,See Other Haktoys HelicoptersSee Other Haktoys...,Haktoys&reg; HAK909 Large 2.4GHz 4CH RC Quadco...,99.99,{'Toys & Games': 40167},http://ecx.images-amazon.com/images/I/418neZCc...,NaN,"[[Toys & Games, Hobbies, RC Vehicles & Parts, ...","{'also_viewed': ['B00D3IN11Q', 'B00EXBDWZC', '..."
336068,B00LRXEK4G,"With a pointed bed sheet hat, robe decorated w...",Teenage Mutant Ninja Turtles Donatello Wizard ...,27.94,{'Toys & Games': 3853},http://ecx.images-amazon.com/images/I/41jIB90F...,Playmates,"[[Toys & Games, Dolls & Accessories, Dolls]]","{'also_bought': ['B00LRWFTQK', 'B00LRX7DXQ', '..."
336069,B00LV83AMU,This Little Mermaid Inspired Slip-On Costume h...,Little Mermaid,NaN,NaN,http://ecx.images-amazon.com/images/I/41VtaqO2...,NaN,[[Toys & Games]],{'also_viewed': ['B00LV83CN2']}
336070,B00LXACM5W,Finding safe water toys for the children is no...,24 Blade Blaster Mini Eliminator Foam Water Gu...,29.99,{'Toys & Games': 62159},http://ecx.images-amazon.com/images/I/51Q7Rjkl...,NaN,"[[Toys & Games, Sports & Outdoor Play, Pools &...","{'also_viewed': ['B00L9EZSQW', 'B00DEM2AC2', '..."


In [127]:
def get_id_from_user_index(index):
    return U_list[index]

def get_id_from_item_index(index):
    return I_list[index]

def get_name_from_id(id):
    return item_desc_df.loc[item_desc_df['asin'] == id].iloc[0]['title']

def get_indices_of_pairs_for_user(user_index):
    return uir[uir[:, 0] == user_index, :]

def get_names_from_ids(ids):
    return list(map(get_name_from_id, map(get_id_from_item_index, ids)))

In [158]:
d = get_indices_of_pairs_for_user(1234)
names = get_names_from_ids(d[:, 1])
pred = mf_model.predict(d[:, :2])
names_ratings = list(zip(names, d[:, 2], pred))
names_ratings

[("LEGO Friends Olivia's Tree House 3065", 5, 5.150842448218809),
 ("LeapFrog LeapPad2 Explorer Kids' Learning Tablet, Green",
  5,
  5.043611775272747),
 ('Nickelodeon Plush Bubble Guppies Puppy', 5, 5.045491788829379),
 ('Nickelodeon Plush Bubble Guppies Molly', 5, 4.985670914638687),
 ('Nickelodeon Plush Bubble Guppies Gil', 5, 4.861390332439493),
 ('Lego Friends Polybag 30106 Emma with Ice Cream Cart Stand',
  5,
  4.717247714005299),
 ('BOOMco. Rapid Madness Blaster', 5, 5.081962183299476)]